In [43]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import math
import os.path

In [44]:
#Mapping Functions

def Mapping_Categ(Cat):
    rtv = Cat

    if (Cat == 'Prepare') or (Cat == 'Training'):
        rtv = 'Prepare Content for future delivry on'
    if (Cat == 'Internal MS Meeting') or (Cat == 'Front'):
        rtv = 'Meeting with'

    return rtv

def Mapping_Subject(Sub):
    rtv = 'Partners'
    rec = re.compile("^\[([a-zA-Z0-9]+)\]")

    try:
        qui = rec.findall(Sub)
        
        if len(qui) > 0:
            rtv = qui[0]
            if rtv.lower() == 'all' : rtv = 'Partners' 
    except:
        pass

    return rtv

def GenReport(Year,Month,df,out_filename):
    
    From = datetime(Year,Month,1)
    if Month < 12:
        To   = datetime(Year,Month+1,1)
    else:
        To   = datetime(Year+1,1,1)
        
    #CleanUp Columns
    df.drop(['Folder Path','Location','DisplayTo','DisplayCc','LegacyFreeBusyStatus','Importance','IsReminderSet','ReminderMinutesBeforeStart','HasAttachments','Preview','Id'],axis=1,inplace=True)

    #Rearrange Date fields
    df['Duration'] = df.End - df.Start
    df['DurationHour'] = df['Duration'].dt.total_seconds().div(3600).astype(float)
    df['DurationHour'] = df['DurationHour'].apply(lambda d: int(math.ceil(d)))
    df['week'] = df['Start'].apply(lambda d: d.isocalendar()[1])
    df['Date'] = pd.to_datetime(df['Start']).dt.normalize()
    df.set_index('Date',inplace=True)
    df = df.sort_values('Date')

    #Slice Data
    df_Z = df.loc[From:To,:].copy()
    
    #Map Activities field
    df_Z['Activity_Purpose'] = df_Z.Categories.map(Mapping_Categ)
    df_Z['Activity_Target'] = df_Z.Subject.map(Mapping_Subject)
    df_Z['Activity'] = df_Z['Activity_Purpose'] + " " + df_Z['Activity_Target']
    df_Z['Day'] = df_Z['Start'].apply(lambda d: d.strftime('%d/%m/%Y'))
    
    #Prepare before writing
    #Remove Admin and Vacances
    df_Z = df_Z.query('Activity_Purpose != "Admin"')
    df_Z = df_Z.query('Activity_Purpose != "Vacances"')
    
    #CleanUp Columns
    final_df = df_Z.drop(['Subject','Duration','Start','End','IsAllDayEvent','Categories','Activity_Purpose','Activity_Target','Duration'],axis=1)
        
    #Agregate by Week / Activity
    grp_by_week_activity = final_df.groupby(['week','Activity']).agg('sum')
    
    #Count the number of RDV
    Nb_rdv = df_Z.Activity_Purpose[df_Z.Activity_Purpose.str.contains('meeting',case=False,na="") == True].count()
    
    #Get the nb of days registered in the month
    ud = np.unique(df_Z.index.date)
    Hour_Max = ud.size*7
    
    #Verify the number of hours
    nb_Hour = grp_by_week_activity.DurationHour.sum()

    #Calculate Delta
    delta = int(Hour_Max - nb_Hour)
    
    if delta > 0:
        #Add activities to fill the gap
        dogf=3
        final_df.reset_index('Date',inplace=True)
        for i in range(delta):
            final_df = final_df.append(pd.DataFrame([[datetime(Year,Month,dogf),1,datetime(Year,Month,dogf).isocalendar()[1],'Prepare Content for future delivry on Partners',datetime(Year,Month,dogf).strftime('%d/%m/%Y')]],
                                            columns=final_df.columns))
        final_df.set_index('Date',inplace=True)
        final_df = final_df.sort_values('Date')
        adjust_str = f'Filling gap : {delta}'
    
        #Agregate by Week / Activity Again after the Touch Up
        grp_by_week_activity = final_df.groupby(['week','Activity']).agg('sum')
        
    
    #Write to out filename
    if os.path.isfile(out_filename):        
        with pd.ExcelWriter(out_filename,mode='a') as writer:  
            grp_by_week_activity.to_excel(writer, sheet_name=f'{From.strftime("%B")} {Year}')            
    else:
        grp_by_week_activity.to_excel(out_filename,sheet_name=f'{From.strftime("%B")} {Year}')
        
    print (f'Nb Hour max for the month : {Hour_Max}')
    print (f'Nb of working days : {ud}')
    print (f'Nb Hour for the month : {nb_Hour}')
    print (f'Nb Meetings : {Nb_rdv}')
    print (adjust_str)
        
    return final_df

In [45]:
YEAR_OF_QUERY=2020
MONTH_OF_QUERY=10
OUT_EXCEL_NAME='Activité-Q2.xlsx'
IMPUT_RAW='Raw-Data-Calendar.xlsx'

In [46]:
for m in [10,11,12]:
    df_raw = pd.read_excel(IMPUT_RAW) 
    GenReport(YEAR_OF_QUERY,m,df_raw,OUT_EXCEL_NAME)

Nb Hour max for the month : 154
Nb Hour for the month : 146
Nb Meetings : 50
Filling gap : 8
Nb Hour max for the month : 147
Nb Hour for the month : 144
Nb Meetings : 48
Filling gap : 3
Nb Hour max for the month : 154
Nb Hour for the month : 145
Nb Meetings : 36
Filling gap : 9


In [29]:
YEAR_OF_QUERY=2021
OUT_EXCEL_NAME='Activité-Q3.xlsx'
IMPUT_RAW='Raw-Data-Calendar.xlsx'

In [18]:
#Query Janvier
MONTH_OF_QUERY=11
df_raw = pd.read_excel(IMPUT_RAW) 
GenReport(YEAR_OF_QUERY,MONTH_OF_QUERY,df_raw,OUT_EXCEL_NAME)

Nb Hour max for the month : 147
Nb Hour for the month : 146
Nb Meetings : 48
Filling gap : 1


,DurationHour,week,Activity,Day
Date,,,,
2020-11-01,1,44,Prepare Content for Future delivery Partner,01/11/2020
2020-11-02,2,45,Meeting with PDM,02/11/2020
2020-11-02,3,45,Meeting with ILIANE,02/11/2020
2020-11-03,1,45,Meeting with INMAC,03/11/2020
2020-11-03,1,45,Meeting with Partners,03/11/2020
...,...,...,...,...
2020-11-30,4,49,Prepare Content for future delivry on INTUNE,30/11/2020
2020-11-30,2,49,Meeting with PDM,30/11/2020
2020-11-30,2,49,Meeting with PEER,30/11/2020
